# 第7回課題
2段階の train, valid, test にわける交差検証の自前の実装
（パッケージにしたり関数にしたりするのは自由です．解答はしていません）

https://scikit-learn.org/stable/modules/cross_validation.html　で様々なパターンについて述べられています

# 注意事項
train, valid, test で取ってくるサンプル（id）が一致しないということを必ず確認してください

In [215]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [216]:
data = load_wine()

In [217]:
wine_df = pd.DataFrame(min_max_scaler.fit_transform(data["data"]), columns=data["feature_names"])
wine_df = pd.concat([wine_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
print(wine_df.shape)
wine_df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0


In [218]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
alcohol                         178 non-null float64
malic_acid                      178 non-null float64
ash                             178 non-null float64
alcalinity_of_ash               178 non-null float64
magnesium                       178 non-null float64
total_phenols                   178 non-null float64
flavanoids                      178 non-null float64
nonflavanoid_phenols            178 non-null float64
proanthocyanins                 178 non-null float64
color_intensity                 178 non-null float64
hue                             178 non-null float64
od280/od315_of_diluted_wines    178 non-null float64
proline                         178 non-null float64
target                          178 non-null int64
dtypes: float64(13), int64(1)
memory usage: 19.5 KB


# モデル（今回は決定木を利用）

In [219]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0)

# seed の設定

In [220]:
seed = 0
random.seed(seed)
np.random.seed(seed=seed)

# k-fold の k は変数で扱うこと

In [221]:
k = 5

# train (valid 含む), test の分割

In [222]:
def train_test_split(df, k):
    # scikit-learnと異なり、説明変数と目的変数の分離を仮定していない
    # df の 1/k を test_df それ以外を train_dfとして返す
    # 余りは test_df に残す
    
    index = list(df.index)
    
    # index が 0 から始まり、 step が常に 1 でない場合を想定していないので例外処理のようなもの
    if index[0] != 0 or index[-1] != df.shape[0]-1:
        print("index error")
        return null
        
    
    train_index = np.random.choice(len(index), df.shape[0]//k*(k-1), replace=False) # 重複させない
    test_index = list(set(index) - set(train_index))
    
    train_df, test_df = df.iloc[train_index], df.iloc[test_index]
    
    return train_df, test_df

In [223]:
train_df, test_df = train_test_split(wine_df, k)

In [224]:
# id の確認
# データ数が一致してかつ重複がなければ良い => train_index と test_index をまとめて sort したものと元の index を比較する
print(wine_df.shape[0] == train_df.shape[0]+test_df.shape[0])
print(list(wine_df.index) == train_df.index.append(test_df.index).sort_values())

True
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  Tr

train_testの分割は成功

# 交差検証
train, valid を分割しつつ valid をテスト

In [225]:
from sklearn.metrics import accuracy_score # 回答はaccuracy だけにしています．ここも特にしていないです

In [226]:
# k回訓練し，valid で評価

train_df は train_test_split ですでにシャッフルされているため、そのまま分割を行う

In [242]:
score = []
print("Validation Score")
for i in range(k-1):
    valid_df = train_df[i*train_df.shape[0]//4:(i+1)*train_df.shape[0]//4] 
    training_df = train_df.drop(labels=valid_df.index)
    
    y_valid, y_training = valid_df["target"], training_df["target"]
    X_valid, X_training = valid_df.drop("target", axis=1), training_df.drop("target", axis=1)
    
    clf.fit(X_training, y_training)
    predict = clf.predict(X_valid)
    score.append(accuracy_score(y_valid, predict))
    print(str(i+1)+"回目: "+str(score[-1]))

print("平均: ",sum(score)/len(score))

Validation Score
1回目: 0.8
2回目: 0.9142857142857143
3回目: 0.8571428571428571
4回目: 0.9428571428571428
平均:  0.8785714285714286


In [17]:
# train すべてで訓練し，test を評価

In [241]:
y_train, y_test = train_df["target"], test_df["target"]
X_train, X_test = train_df.drop("target", axis=1), test_df.drop("target", axis=1)

In [243]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Test score: ", accuracy_score(y_test, y_pred))

Test score:  0.9210526315789473
